<a href="https://colab.research.google.com/github/nanac0n/2024ML_Are_U_T/blob/main/Project_Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install transformers==4.28.1
!pip install scikit-learn==1.2.2
!pip install torch==2.0.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 25.7 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.1
    Uninstalling transformers-4.41.1:
      Successfully uninstalled transformers-4.41.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 76.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 72.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 104.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 3.4 MB/s eta 0:00:00


In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset, DataLoader

# 데이터 로드 및 전처리
file_path = '/content/drive/MyDrive/adasyn.csv'  # 실제 파일 경로를 지정하세요.
df = pd.read_csv(file_path)

df['type'] = df['type'].replace({
    'INTJ': 'T', 'INTP': 'T', 'ENTJ': 'T', 'ENTP': 'T',
    'ISTJ': 'T', 'ISTP': 'T', 'ESTJ': 'T', 'ESTP': 'T',
    'INFJ': 'F', 'INFP': 'F', 'ENFJ': 'F', 'ENFP': 'F',
    'ISFJ': 'F', 'ISFP': 'F', 'ESFJ': 'F', 'ESFP': 'F'
})

# 데이터셋 크기를 줄이기 (예: 10%만 사용)
df = df.sample(frac=0.1, random_state=42)

texts = df['posts'].values
labels = df['type'].values

# 라벨 인코딩 (T -> 0, F -> 1)
label_dict = {'T': 0, 'F': 1}
labels = np.array([label_dict[label] for label in labels])

# 학습 및 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

# DistilBERT 토크나이저
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

class MBTIDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [22]:
# 학습 인자 설정
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,  # 에포크 수를 늘리기
    per_device_train_batch_size=16,  # 배치 크기 줄이기 (GPU 메모리 상황에 맞게 조정)
    per_device_eval_batch_size=16,
    learning_rate=2e-5,  # 학습률 조정
    warmup_steps=200,  # 워밍업 단계 수 늘리기
    weight_decay=0.01,
    evaluation_strategy="epoch",
    logging_dir='./logs',
    load_best_model_at_end=True,
    metric_for_best_model='eval_accuracy',
    save_strategy='epoch'  # Change save_strategy to epoch
)

# 평가 지표 계산 함수 정의
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {
        'accuracy': (predictions == labels).mean()
    }

# Trainer 정의
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics # 추가: 평가 지표 계산 함수
)

In [23]:
# 모델 학습
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.264800,0.621207,0.731926
2,0.284900,0.732108,0.748560
3,0.185000,0.969375,0.736084


TrainOutput(global_step=2346, training_loss=0.24092814745500568, metrics={'train_runtime': 847.1544, 'train_samples_per_second': 44.28, 'train_steps_per_second': 2.769, 'total_flos': 970530675465600.0, 'train_loss': 0.24092814745500568, 'epoch': 3.0})

In [24]:
# 모델 평가
eval_result = trainer.evaluate()
print(f"Test Accuracy: {eval_result['eval_accuracy']:.4f}")

# 예측 함수
def predict_mbti(text):
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=100,
        return_token_type_ids=False,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt',
    )
    # Move input tensors to GPU
    input_ids = encoding['input_ids'].to(model.device)  # Move input_ids to GPU
    attention_mask = encoding['attention_mask'].to(model.device)  # Move attention_mask to GPU
    outputs = model(input_ids, attention_mask=attention_mask)
    _, prediction = torch.max(outputs.logits, dim=1)
    return 'T' if prediction.item() == 0 else 'F'

Test Accuracy: 0.7486


In [25]:
model.save_pretrained("/content/drive/MyDrive/Bert_model.h5")

In [26]:
# 예측 예시
sample_text = "I'm so sad"
predicted_type = predict_mbti(sample_text)
print(f'Predicted MBTI type: {predicted_type}')

Predicted MBTI type: F
